* Lucas Montero - 201473588-k
* Miguel Bustamante - 201610501-8

# Sistema recomendacion


In [1]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer 
import numpy as np
from nltk.corpus import wordnet as wn
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
df1=pd.read_csv('./data/tmdb_5000_credits.csv') #Guardamos una base de datos en df1
df2=pd.read_csv('./data/tmdb_5000_movies.csv')  #Guardamos la otra en df2
df1.columns = ['id','tittle','cast','crew']     #Renombramos las columnas de df1
df2= df2.merge(df1,on='id')                     #Fusionamos df2 con df1

In [2]:
df = df2    #Cambiamos de nombre a df ya que solo tenemos una base de datos
df.head(7)  #Visualizamos las primeras 7 filas

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,tittle,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
5,258000000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",http://www.sonypictures.com/movies/spider-man3/,559,"[{""id"": 851, ""name"": ""dual identity""}, {""id"": ...",en,Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,115.699814,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,139.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,The battle within.,Spider-Man 3,5.9,3576,Spider-Man 3,"[{""cast_id"": 30, ""character"": ""Peter Parker / ...","[{""credit_id"": ""52fe4252c3a36847f80151a5"", ""de..."
6,260000000,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...",http://disney.go.com/disneypictures/tangled/,38757,"[{""id"": 1562, ""name"": ""hostage""}, {""id"": 2343,...",en,Tangled,When the kingdom's most wanted-and most charmi...,48.681969,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,1

In [3]:
df.shape #Obtenemos las dimensiones de df

(4803, 23)

# Limpieza de información, Tokenizacion y demás

Con sklearn se puede hacer casi todo el proceso dentro de la funcion de vectorización. No obstante se prefirió hacer esto a mano para hacer steaming (mejorando un poco los resultados) y además retornando el mismo formato de lista de strings para vectorizar.

In [4]:
df2['overview'] = df2['overview'].fillna('') #Cambiamos los NaN por string vacio

In [11]:
sentences = df["overview"].values.astype(str)   #Guardamos 'overview' de la base de datos
                                                #en sentencias y pasamos todo a string
obj = []                       #Creamos una lista vacía
steem= PorterStemmer()         #Cargamos PorterStemmer en Steem
for a in sentences:            #Recorremos las sentencias
    a=a.lower()                #Ponemos todas las palabras en minusculas
    a=a.replace(".","")        #Eliminamos los puntos
    a=a.replace(",","")        #Eliminamos los comas
    a=a.replace(":","")        #Eliminamos los doble punto
    a=a.replace(";","")        #Eliminamos los puntos y comas
    a=a.replace("-","")        #Eliminamos los guiones
    a=a.replace("/","")        #Eliminamos los slashes
    a=nltk.word_tokenize(a)    #Separamos las palabras en la sentencia
    words=""                   #Creamos una palabra 'vacia'
    for w in a:                                 #Recorremos la sentencia
        if w not in stopwords.words('english'): #Si no es un conector, entonces:
            words+=steem.stem(w)+" "            #Guardamos las palabra normalizada en words
            
    obj.append(words)          #Guardamos words en la lista obj
print(obj)                     #Printeamos la lista

['22nd centuri parapleg marin dispatch moon pandora uniqu mission becom torn follow order protect alien civil ', 'captain barbossa long believ dead come back life head edg earth turner elizabeth swann noth quit seem ', 'cryptic messag bond ’ past send trail uncov sinist organ battl polit forc keep secret servic aliv bond peel back layer deceit reveal terribl truth behind spectr ', "follow death district attorney harvey dent batman assum respons dent 's crime protect late attorney 's reput subsequ hunt gotham citi polic depart eight year later batman encount mysteri selina kyle villain bane new terrorist leader overwhelm gotham 's finest dark knight resurfac protect citi brand enemi ", "john carter warweari former militari captain 's inexplic transport mysteri exot planet barsoom ( mar ) reluctantli becom embroil epic conflict 's world brink collaps carter rediscov human realiz surviv barsoom peopl rest hand ", 'seemingli invinc spiderman goe allnew crop villain – includ shapeshift sand

# Vectorizacion

In [12]:
vectorizer = TfidfVectorizer()               #Guardamos TfidfVectorizer en vectorizer
descripcion = vectorizer.fit_transform(obj)  #Vectorizamos nuestra lista en base a recurrencia
print(descripcion[0])                        #Printiamos para ver nuestro resultado

  (0, 2992)	0.23543066583389285
  (0, 732)	0.20918511524573216
  (0, 11905)	0.1953417785382612
  (0, 10850)	0.18178525087478506
  (0, 5793)	0.17060082296895818
  (0, 15221)	0.26049699134299037
  (0, 1626)	0.1345928787757917
  (0, 9870)	0.1878614569169705
  (0, 15724)	0.2556261556427692
  (0, 11076)	0.3114514381968726
  (0, 9996)	0.2927599070741794
  (0, 4361)	0.28278681757672935
  (0, 9350)	0.26225701466165824
  (0, 11108)	0.3632585813636025
  (0, 2739)	0.21122771836468612
  (0, 256)	0.32536601403654747


Nota: Curiosamente el print no muestra los ceros.

In [18]:
int(np.random.random_sample(range(len(df))))

ValueError: maximum supported dimension for an ndarray is 32, found 4803

# Top 10

Bueno, acá dejaré comentada la opción para colocar un input a mano o escogiendo algun indice.

In [23]:
titulo_ejemplo = df['title'][np.random.randint(len(df))]  #Escogemos un título aleatorio
df_ex =(df.index[df['title'] == titulo_ejemplo]).tolist() #Cargamos fila del título
                                                          #aleatorio en una lista

In [24]:

print("Obteniendo top 10 del titulo:",titulo_ejemplo,"\n")
top10_I=[0,0,0,0,0,0,0,0,0,0]               #Creamos dos listas de 0s
top10_d=[0,0,0,0,0,0,0,0,0,0]
index_count=-1                              #Creamos un contador y lo inicializamos en -1
for i in descripcion:                       #Recorremos las descripciones de las películas
    index_count+=1                          #Sumamos 1 al contador
    simi=-3                                 #Inicializamos la similaridad
    res = descripcion[df_ex[0]]             #Guardamos la descipción de la pelicula ejemplo
    if df_ex[0] == index_count:             #Si revisamos la misma pelicula:
        continue                            #Hacemos nada
    else:                                   #Si no:
        simi = cosine_similarity(res,i)     #Comparamos las semejanzas 
        for k in range(len(top10_d)):       #Recorremos el top 10
            
            if simi>top10_d[k]:             #Si la semejanza es mayor que alguno:
                top10_d[k]=simi             #Guardamos la nueva película
                top10_d.sort(reverse=True)  #Ordenamos el top10
                ind = top10_d.index(simi)   #Guardamos la posición de la nueva película
                top10_I[ind]=index_count    #Guardamos esa posición en la lista de top10
                break
            else:                           #Si no:
                continue                    #Saltamos a la siguiente película
    
print_count=1                               #Inicializamos una variable auxiliar
for j in range(10):                         #Recorremos el top10
    print(print_count,")",df["title"][top10_I[j]], ". Con similitud:",top10_d[j]," \n")
                                            #Printeamos
    print_count+=1

Obteniendo top 10 del titulo: Buffalo Soldiers 

1 ) Miracle at St. Anna . Con similitud: [[0.18249617]]  

2 ) Among Giants . Con similitud: [[0.17340176]]  

3 ) Frozen River . Con similitud: [[0.15722772]]  

4 ) Cinco de Mayo: La Batalla . Con similitud: [[0.12926299]]  

5 ) Shopgirl . Con similitud: [[0.11694373]]  

6 ) Saints and Soldiers . Con similitud: [[0.10330768]]  

7 ) Slam . Con similitud: [[0.09991886]]  

8 ) Dead Snow . Con similitud: [[0.09878746]]  

9 ) Waltz with Bashir . Con similitud: [[0.09068516]]  

10 ) Dude Where's My Dog? . Con similitud: [[0.08604248]]  

